In [10]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

In [11]:
one_hot_df = pd.read_csv('./Data/one_hot_df.csv')
label_df = pd.read_csv('./Data/label_df.csv')
y = one_hot_df['SalePrice']
X = one_hot_df.drop(['SalePrice', 'Unnamed: 0'], axis =1)
label_X = label_df.drop(['SalePrice', 'Unnamed: 0'], axis =1)

## Linear Regression

In [12]:
lr_model = LinearRegression()
metrics = cross_validate(lr_model, X,y,cv = 5, scoring = ('neg_root_mean_squared_error'))
lr_cv = -metrics['test_score'].mean()
print('CV PREDICTION ERROR FOR MULTIVARIATE LR')
print(lr_cv)

CV PREDICTION ERROR FOR MULTIVARIATE LR
2656881023.263894


In [13]:
lr_model.fit(X,y)
lr_model.score(X,y)

0.9258276520543647

The linear regression model gives a very high R squared of 0.926, but this is expected because we have included almost all features inside our regression model. Looking for adjusted r squared metric. We have 272 predictor variables in our model. 

The CV prediction error we are getting is abnormally high. We need to investigate this. Does not make sense, because it is a log(sales price) the RMSE cannot go into the millions

## Ridge Regression

In [14]:
param_list = []
start = 10
for i in range(30):
    param_list.append(start)
    start = start + 0.25

parameters = {'alpha': param_list} 
rr = Ridge()
metrics = cross_validate(rr, X, y, cv = 5, scoring = ('neg_root_mean_squared_error'))
print('CV PREDICTION ERROR FOR UNTUNED RR')
print(-metrics['test_score'].mean())

CV PREDICTION ERROR FOR UNTUNED RR
0.14151972402903668


Tuning the ridge regression model

In [15]:
tuned_rr = GridSearchCV(rr, parameters, scoring = 'neg_root_mean_squared_error', cv = 5)
tuned_rr.fit(X,y)
print('CV PREDICTION ERROR FOR TUNED RR')
print(-tuned_rr.best_score_)
print(tuned_rr.best_params_)

CV PREDICTION ERROR FOR TUNED RR
0.13646741581659594
{'alpha': 15.0}


We find the value of alpha that gives us the lowest cross validation prediction error. The alpha 

## Lasso Regression

In [16]:
param_list = []
start = 0.0001
for i in range(30):
    param_list.append(start)
    start = start + 0.0001
parameters2 = {'alpha': param_list}
lasso = Lasso()
lasso.fit(X,y)
metrics = cross_validate(lasso, X,y, cv = 5, scoring = ('neg_root_mean_squared_error'))
print('SCORE FOR UNTUNED LASSO')
print(-metrics['test_score'].mean())

SCORE FOR UNTUNED LASSO
0.3992282792085989


Tuning the lasso regression model

In [17]:
tuned_lasso = GridSearchCV(lasso, parameters2, scoring = 'neg_root_mean_squared_error', cv = 5)
tuned_lasso.fit(X,y)
best_param = tuned_lasso.best_params_['alpha']
lasso_tuned = Lasso(alpha = best_param)
lasso_tuned.fit(X,y)
metrics = cross_validate(lasso_tuned, X,y, cv = 5, scoring = ('neg_root_mean_squared_error'))


Analysis of tuned lasso regression model

In [18]:
print('BASED ON CV, WE FOUND OPTIMAL ALPHA TO BE:')
print(best_param)
print(-tuned_lasso.best_score_)
print()
model_coefs = list(lasso_tuned.coef_)

good_features = []
for i in range(len(model_coefs)):
    if model_coefs[i]> 0:
        good_features.append(list(X.columns)[i])

print('\n'+'FEATURES WITH NON ZERO COEFFICIENTS FOR LASSO')
print(good_features)

BASED ON CV, WE FOUND OPTIMAL ALPHA TO BE:
0.0005
0.13390462926904387


FEATURES WITH NON ZERO COEFFICIENTS FOR LASSO
['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'ScreenPorch', 'MSSubClass_20', 'MSSubClass_50', 'MSZoning_RL', 'LotShape_IR2', 'LotConfig_Corner', 'LotConfig_CulDSac', 'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_Crawfor', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_Somerst', 'Neighborhood_StoneBr', 'Condition1_Norm', 'Condition1_RRAn', 'BldgType_1Fam', 'Exterior1st_BrkFace', 'Exterior1st_MetalSd', 'Exterior1st_Plywood', 'Exterior1st_VinylSd', 'Exterior2nd_VinylSd', 'Exterior2nd_Wd Sdng', 'ExterQual_Gd', 'ExterCond_TA', 'Foundation_PConc', 'BsmtQual_Ex', 'BsmtCond_TA', 'BsmtExposure_Gd', 'BsmtFinType1_

In [19]:
for i in good_features:
    if '_' in i:
        print(i, end = ', ')

MSSubClass_20, MSSubClass_50, MSZoning_RL, LotShape_IR2, LotConfig_Corner, LotConfig_CulDSac, Neighborhood_BrkSide, Neighborhood_ClearCr, Neighborhood_Crawfor, Neighborhood_NoRidge, Neighborhood_NridgHt, Neighborhood_Somerst, Neighborhood_StoneBr, Condition1_Norm, Condition1_RRAn, BldgType_1Fam, Exterior1st_BrkFace, Exterior1st_MetalSd, Exterior1st_Plywood, Exterior1st_VinylSd, Exterior2nd_VinylSd, Exterior2nd_Wd Sdng, ExterQual_Gd, ExterCond_TA, Foundation_PConc, BsmtQual_Ex, BsmtCond_TA, BsmtExposure_Gd, BsmtFinType1_ALQ, BsmtFinType1_GLQ, BsmtFinType2_Unf, Heating_GasW, HeatingQC_Ex, HeatingQC_Gd, Electrical_FuseA, KitchenQual_Ex, Functional_Typ, GarageType_Attchd, GarageType_Detchd, GarageQual_Gd, GarageCond_TA, PavedDrive_Y, MoSold_5, MoSold_6, MoSold_7, SaleCondition_Normal,

We perform feature selection based on the coefficients of our lasso regression model, narrowing our features down from 273 to 66

The value of alpha that gives us the lowest CV prediction error is 0.0004. This alpha value is very low: when alpha is low it means that the result (model coefficients) become similar to that of the linear regression model. Because the penalty that is placed on the coefficients in the optimisation function is lower

Interesting selection of features: Based on the one hot encoding, not all were important. For example, for months, if variable was in month 5,6,7 seemed to have an impact. This incentivizes us to think that perhaps doing some feature engineering might improve the results. Change the classes found within some of the features which are categorical variables.

## Reworking based on feature selection

In [20]:
new_feature_set = X[good_features]
lr_model_selected = LinearRegression()
metrics = cross_validate(lr_model_selected, new_feature_set, y, cv = 5,scoring = ('neg_root_mean_squared_error'))
print('CV PREDICTION ERROR FOR NEW FEATURE SET MULTIVARIATE LR')
-metrics['test_score'].mean()

CV PREDICTION ERROR FOR NEW FEATURE SET MULTIVARIATE LR


0.13631552802441838

Running a linear regression model based on feature selection with our lasso regression model helps us derive a significantly lower CV prediction error at 0.136350

In [ ]:
param_list = []
start = 4
for i in range(30):
    param_list.append(start)
    start = start + 0.1
#     start = start +

parameters = {'alpha': param_list} 
rr = Ridge()
metrics = cross_validate(rr, X[good_features], y, cv = 5, scoring = ('neg_root_mean_squared_error'))
print('CV PREDICTION ERROR FOR UNTUNED RR')
print(-metrics['test_score'].mean())

tuned_rr = GridSearchCV(rr, parameters, scoring = 'neg_root_mean_squared_error', cv = 5)
tuned_rr.fit(X[good_features],y)
print('CV PREDICTION ERROR FOR TUNED RR')
print(-tuned_rr.best_score_)
print(tuned_rr.best_params_)

We see little difference between our original ridge regression model and the new ridge regression model. CV prediction model is very similar, slight improvement when we incorporate feature selection

In [ ]:
param_list = []
start = 0.0001
for i in range(30):
    param_list.append(start)
    start = start + 0.0001
parameters2 = {'alpha': param_list}
lasso = Lasso()
lasso.fit(new_feature_set,y)
metrics = cross_validate(lasso, new_feature_set,y, cv = 5, scoring = ('neg_root_mean_squared_error'))
print('SCORE FOR UNTUNED LASSO')
print(-metrics['test_score'].mean())

In [ ]:
tuned_lasso = GridSearchCV(lasso, parameters2, scoring = 'neg_root_mean_squared_error', cv = 5)
tuned_lasso.fit(new_feature_set,y)
best_param = tuned_lasso.best_params_['alpha']
lasso_tuned = Lasso(alpha = best_param)
lasso_tuned.fit(X,y)
metrics = cross_validate(lasso_tuned, new_feature_set,y, cv = 5, scoring = ('neg_root_mean_squared_error'))


In [ ]:
print('BASED ON CV, WE FOUND OPTIMAL ALPHA TO BE:')
print(best_param)
print(-tuned_lasso.best_score_)
print()
model_coefs = list(lasso_tuned.coef_)

We found that the CV prediction error does not change after we include feature selection. This is because of the mechanism of the lasso regression, whereby insignificant variables are already shrunk towards 0

## Random Forest

In [25]:
rf = RandomForestRegressor(n_estimators = 500, max_leaf_nodes = 16, n_jobs = -1, oob_score = True, bootstrap = True)
rf.fit(new_feature_set, y)
print(rf.get_params())
print('\n This is the oob score: ', rf.oob_score_)
rf_features = []

for name, score in zip(list(label_X.columns),rf.feature_importances_):
    if score > 0:
#         print(name,score)
        rf_features.append((name,score))


{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': 16, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 500, 'n_jobs': -1, 'oob_score': True, 'random_state': None, 'verbose': 0, 'warm_start': False}

 This is the oob score:  0.8067589925893359


In [ ]:
rf = RandomForestRegressor(n_estimators = 500, max_leaf_nodes = 16, n_jobs = -1, oob_score = True, bootstrap = True)
rf.fit(new_feature_set, y)
print(rf.get_params())
print('\n This is the oob score: ', rf.oob_score_)
rf_features = []

for name, score in zip(list(label_X.columns),rf.feature_importances_):
    if score > 0:
#         print(name,score)
        rf_features.append((name,score))

In [30]:
# rf_features.sort(key = lambda x:x[1], reverse = True)
# rf_features

In [27]:
print('LASSO REGRESSION FEATURES SELECTED: \n', good_features)
print()
print('FEATURES USED TO SPLIT RF: \n', rf_features)

LASSO REGRESSION FEATURES SELECTED: 
 ['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'ScreenPorch', 'MSSubClass_20', 'MSSubClass_50', 'MSZoning_RL', 'LotShape_IR2', 'LotConfig_Corner', 'LotConfig_CulDSac', 'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_Crawfor', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_Somerst', 'Neighborhood_StoneBr', 'Condition1_Norm', 'Condition1_RRAn', 'BldgType_1Fam', 'Exterior1st_BrkFace', 'Exterior1st_MetalSd', 'Exterior1st_Plywood', 'Exterior1st_VinylSd', 'Exterior2nd_VinylSd', 'Exterior2nd_Wd Sdng', 'ExterQual_Gd', 'ExterCond_TA', 'Foundation_PConc', 'BsmtQual_Ex', 'BsmtCond_TA', 'BsmtExposure_Gd', 'BsmtFinType1_ALQ', 'BsmtFinType1_GLQ', 'BsmtFinType2_Unf', 'Heating_GasW', 'HeatingQC_Ex', 'H

Running our analysis we can see very similar features being used to split our decision trees, and the features selected by the LASSO regression model. How about we try to do feature selection using the lasso regression?

We may also want to see the difference between using different encodings for our random forest model - whether we want to use label encoding or use one hot encoding. See below: it seems like using label encoding significantly improves the performance of your model because the MSE is lower.

## Tuning of RF model

Using the OOB score to evaluate the random forest model gives us an extremely high OOB r^2 score. 
Based on the model, we also see that many features have very low importance, and this incentivizes us to revisit the variable selection methodology that we adopted earlier.

We try hyperparameter search using random hyperparameter grid, so that we can find optimum hyperparamters for our random forest model

In [28]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num =10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10,110,num =11)]
max_depth.append(None)
min_samples_split = [2,5,10]
min_samples_leaf = [1,2,4]
bootstrap = [True]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [29]:
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 25, cv =5, verbose = 2, random_state = 42, n_jobs = -1, scoring = 'neg_root_mean_squared_error')
rf_random.fit(new_feature_set, y)
print('THIS IS THE BEST SCORE')
print(-rf_random.best_score_)
print('THIS IS THE BEST PARAMS')
print(rf_random.best_params_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   14.0s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:  3.3min finished
THIS IS THE BEST SCORE
0.1393654602033622
THIS IS THE BEST PARAMS
{'n_estimators': 800, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': False}


In [ ]:
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 25, cv =5, verbose = 2, random_state = 42, n_jobs = -1, scoring = 'neg_root_mean_squared_error')
rf_random.fit(label_X, y)
print('THIS IS THE BEST SCORE')
print(-rf_random.best_score_)
print('THIS IS THE BEST PARAMS')
print(rf_random.best_params_)

We narrowed down the scope of the analysis. We can now adopt grid search with cross validation to do a more in depth search of the optimum hyperparameters

Using our paramaters, we derived CV prediction error of 0.13586, based on 5 fold CV.

In [ ]:

param_grid = {
    'max_depth': [20,25,15],
    'min_samples_leaf': [1],
    'min_samples_split': [2],
    'max_features' : ['sqrt'],
    'n_estimators': [800,850, 750]
#     'bootstrap': [False]
}


In [ ]:
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 5, n_jobs= -1, verbose = 2, scoring = 'neg_root_mean_squared_error')
grid_search.fit(X,y)
# print(grid_search.best_params_)
print('SCORE FOR RF')
print(-grid_search.best_score_)

## Other concerns and questions

1. Incorporating variable selection into our model 
2. On the internet, everybody is approaching with validation set approach so that they can check whether there is overfitting. How can we check for overfitting when we use a random forest model?
3. Why do people still use the validation set approach when using cross validation is supposed to help with this? You check across the different parameters to see which set of parameters gives you the lowest CV error